<a href="https://colab.research.google.com/github/hamiin47/OB-session/blob/main/Fri_exercise_0523.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. CIFAR-10 데이터셋을 이용한 CNN 모델

## Step 1. 필요한 라이브러리 임포트

In [1]:
# 필요한 라이브러리 불러오기
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

In [2]:
# 디바이스 설정 (GPU 사용 가능하면 GPU, 없으면 CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## Step 2. CIFAR-10 데이터셋 다운로드

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 정규화 추가
])

# 데이터셋 로드
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)


100%|██████████| 170M/170M [00:01<00:00, 101MB/s]


##Step 3. DataLoader 설정

In [4]:
# DataLoader를 사용하여 배치 크기 64로 데이터를 로드하세요.
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64)

## Step 4. CNN 모델 정의

In [6]:
# CNN 모델 정의

# CNN 모델 정의

class CIFAR10_CNN(nn.Module):
    def __init__(self):
        super(CIFAR10_CNN, self).__init__()

        # 첫 번째 합성곱 층
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)  # 배치 정규화 채널 수 입력

        # 두 번째 합성곱 층
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.bn2 = nn.BatchNorm2d(64)

        # 세 번째 합성곱 층
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.bn3 = nn.BatchNorm2d(128)
        # 풀링 층 (MaxPooling)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully Connected Layer
        self.fc1 = nn.Linear(128 * 2* 2, 512)  # CIFAR-10은 32x32 크기
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        # 활성화 함수 적용
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))

        x = x.view(x.size(0), -1)  # Flatten

        x = F.relu(self.fc1(x))  # Fully Connected 활성화 함수 추가
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


## Step 5. 모델 학습 설정

In [7]:
# 모델을 초기화하세요.
model = CIFAR10_CNN()

# 손실 함수로 CrossEntropyLoss를 사용하고 옵티마이저는 Adam을 사용하세요.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.01)

## Step 6. 모델 학습 루프

In [8]:
# 모델 학습을 위한 루프를 작성하세요. 에폭 수는 3로 설정합니다.
# 런타임 약 5분
num_epochs = 3
for epoch in range(num_epochs):
   for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # 모델을 통해 예측값을 계산
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs, labels)

        # 경사 초기화
        optimizer.zero_grad()

        # 역전파
        loss.backward()
        # 최적화
        optimizer.step()
   print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/3], Loss: 1.7281
Epoch [2/3], Loss: 1.4510
Epoch [3/3], Loss: 1.4541


## Step 7. 테스트 정확도 평가

In [10]:
# 모델의 성능 평가
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted==labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 60.37%


# 2. MNIST 데이터셋 분류 모델




## Step 1. 필요한 라이브러리 임포트

In [11]:
# 필요한 라이브러리 불러오기
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

In [12]:
# 디바이스 설정 (GPU 사용 가능하면 GPU, 없으면 CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## Step 2. MNIST 데이터셋 다운로드

In [13]:
# MNIST 데이터셋을 다운로드하고, 텐서로 변환할 수 있도록 필요한 전처리 함수를 추가하세요.
transform = transforms.Compose([transforms.ToTensor()])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)


100%|██████████| 9.91M/9.91M [00:00<00:00, 129MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 31.4MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 41.3MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.24MB/s]


##Step 3. DataLoader 설정

In [14]:
# DataLoader를 사용하여 배치 크기 64로 데이터를 로드하세요.
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64)

## Step 4. CNN 모델 정의

In [15]:
# CNN 모델 정의
class MNIST_CNN(nn.Module):
  def __init__(self):
    super(MNIST_CNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size = 3, padding = 1)
    self.bn1 = nn.BatchNorm2d(32)
    self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3)
    self.bn2 = nn.BatchNorm2d(64)
    self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)
    self.fc1 = nn.Linear(64*6*6, 600)
    self.fc2 = nn.Linear(600, 120)
    self.fc3 = nn.Linear(120, 10)
    self.dropout = nn.Dropout2d(0.25)


  def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## Step 5. 모델 학습 설정

In [16]:
# 모델을 초기화하세요.
model = MNIST_CNN().to(device)

# 손실 함수로 CrossEntropyLoss를 사용하고 옵티마이저는 Adam을 사용하세요.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

## Step 6. 모델 학습 루프

In [17]:
# 모델 학습을 위한 루프를 작성하세요. 에폭 수는 3로 설정합니다.
# 런타임 약 5분
num_epochs = 3
for epoch in range(num_epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # 이미지 차원 변환 (100, 1, 28, 28) 형태 유지
        images = Variable(images)

        # 모델을 통해 예측값을 계산
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs, labels)

        # 경사 초기화
        optimizer.zero_grad()

        # 역전파
        loss.backward()

        # 최적화
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:1538: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Epoch [1/3], Loss: 0.0330
Epoch [2/3], Loss: 0.0155
Epoch [3/3], Loss: 0.0946


## Step 7. 테스트 정확도 평가

In [18]:
# 모델의 성능 평가
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 96.99%
